In [199]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Font
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.styles import Border, Side
from openpyxl.styles import numbers

In [213]:
#Reading the .CSV files to Python.
sales = pd.read_csv("C:\\Users\\HP\\JJ_projects\\data.csv")
purchase = pd.read_csv("C:\\Users\\HP\\JJ_projects\\Purchase.csv", encoding="ISO-8859-1")
rebate = pd.read_csv("C:\\Users\\HP\\JJ_projects\\Rebate.csv", encoding="ISO-8859-1")

In [201]:
#Trimming the whitespaces from the Code in Sales Data.
sales.columns = sales.columns.str.strip()

#Splitting the merged columns to seperate Product_ID and Descritpion in Sales Data.
sales[['Product_ID', 'Description']] = sales['Item_Code_with_Name'].str.split('-', n=1, expand=True)

#Converting Quantity column in Purchase data to Numeric also catching non-numeric data type.
purchase['Quantity'] = pd.to_numeric(purchase['Quantity'], errors='coerce')

#Converting Date column in Sales data to Date format.
sales['YYYY-MM-DD'] = pd.to_datetime(sales['YYYY-MM-DD'])


In [202]:
# Convert dates with day-first format
rebate['from_date'] = pd.to_datetime(rebate['From date'], dayfirst=True, errors='coerce')
rebate['to_date'] = pd.to_datetime(rebate['To date'], dayfirst=True, errors='coerce')
purchase['P_Date'] = pd.to_datetime(purchase['Delivery Date'], dayfirst=True, errors='coerce')
sales['YYYY-MM-DD'] = pd.to_datetime(sales['YYYY-MM-DD'], dayfirst=True, errors='coerce')

#print(sales)
#sales.to_excel("output.xlsx", index=False)

In [203]:
# Define the Month date range (e.g., for 2024)
feb_start = pd.Timestamp("2025-03-01")
feb_end = pd.Timestamp("2025-03-31")

In [204]:
# Filter for rebates active during February
feb_rebates = rebate[
    (rebate['from_date'] <= feb_end) & 
    (rebate['to_date'] >= feb_start)
]

rebate['from_date'] = rebate['from_date'].apply(lambda x: max(x, feb_start))
rebate['to_date'] = rebate['to_date'].apply(lambda x: min(x, feb_end))


In [205]:
sales['Product_ID'] = sales['Product_ID'].astype(str).str.strip()
rebate['Item number'] = rebate['Item number'].astype(str).str.strip()


In [206]:
#Process "Buying in" rebate rows
buying_in_rows = purchase.merge(
    rebate[rebate['RebateType'] == 'Buying in'], 
    left_on='Item No', 
    right_on='Item number',
    how='outer'
)

# Filter for valid rebate periods
buying_in_valid = buying_in_rows[
    (buying_in_rows['P_Date'] >= buying_in_rows['from_date']) &
    (buying_in_rows['P_Date'] <= buying_in_rows['to_date'])
].copy()

buying_in_valid['Final_Quantity'] = pd.to_numeric(buying_in_valid['Quantity'], errors='coerce')
buying_in_valid['Date'] = buying_in_valid['P_Date']

buying_in_valid.columns = buying_in_valid.columns.str.strip()


In [207]:
#Process "Selling out" rebate rows
sales_out_rows = sales.merge(
    rebate[rebate['RebateType'] == 'Selling out'], 
    left_on='Product_ID', 
    right_on='Item number',
    how='outer'
)


# Filter for valid rebate periods
sales_out_valid = sales_out_rows[
    (sales_out_rows['YYYY-MM-DD'] >= sales_out_rows['from_date']) &
    (sales_out_rows['YYYY-MM-DD'] <= sales_out_rows['to_date'])
].copy()



sales_out_valid['Final_Quantity'] = pd.to_numeric(sales_out_valid['Quantity Ordered'], errors='coerce').fillna(0)
sales_out_valid['Date'] = sales_out_valid['YYYY-MM-DD']

sales_out_valid.columns = sales_out_valid.columns.str.strip()




In [208]:
#Remove any Selling out rebate rows without actual sales
sales_out_valid = sales_out_valid[sales_out_valid['Final_Quantity'] > 0]


In [209]:

#Ensure Buying in values do not appear in "Selling out"
sales_out_valid = sales_out_valid[['Supplier account','Name','Product name','Amount','Item number', 'from_date', 'to_date', 'Final_Quantity','Supplier rebate','RebateType']]
buying_in_valid = buying_in_valid[['Supplier account','Name','Product name','Amount','Item number', 'from_date', 'to_date', 'Final_Quantity','Supplier rebate','RebateType']]



In [210]:

# Combine the filtered datasets
combined_rows = pd.concat([buying_in_valid, sales_out_valid], ignore_index=True)


#Group by 'Item number', 'from_date', 'to_date' and sum 'Final_Quantity'
result = combined_rows.groupby(['Supplier account','Name','Product name','Amount','Item number', 'from_date', 'to_date','Supplier rebate','RebateType'], as_index=False)['Final_Quantity'].sum()


In [211]:
# Calculate Total Amount
result['Total_Amount'] = result['Final_Quantity'] * result['Amount']

final_df = result[
    (result['Final_Quantity'] > 0)
][[
    'Supplier account', 'Name', 'Item number', 'Product name', 'Amount', 'Final_Quantity', 'Total_Amount','from_date', 'to_date','Supplier rebate', 'RebateType'
]]


In [212]:

df = pd.DataFrame(final_df)

# Assuming 'df' is your DataFrame
df['Total_Amount'] = pd.to_numeric(df['Total_Amount'], errors='coerce').fillna(0)

# Sort and prepare output DataFrame
output_frames = []

for name, group in df.groupby('Name'):
    # Add header row for each group
    output_frames.append(pd.DataFrame([df.columns.tolist()], columns=df.columns))
    
    # Add group data
    output_frames.append(group)
    
    # Add total row under Total_Amount
    total_row = [''] * len(df.columns)
    total_row[df.columns.get_loc('Total_Amount')] = group['Total_Amount'].sum()
    output_frames.append(pd.DataFrame([total_row], columns=df.columns))
    
    # Add 2 blank rows after each group
    output_frames.append(pd.DataFrame([[''] * len(df.columns)] * 2, columns=df.columns))

# Combine everything into one final DataFrame
final_output = pd.concat(output_frames, ignore_index=True)

# Export to Excel
output_file = "final_output_with_totals.xlsx"
final_output.to_excel(output_file, index=False, header=False)

# Load workbook for formatting
wb = load_workbook("final_output_with_totals.xlsx")
ws = wb.active

# Apply bold formatting to all header rows
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    if all(cell.value == col_name for cell, col_name in zip(row, df.columns.tolist())):
        for cell in row:
            cell.font = Font(bold=True)

# Define yellow fill style
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Find the 'Total_Amount' column
total_amount_col = list(df.columns).index('Total_Amount') + 1

# Loop through all rows and detect subtotals
for row in range(2, ws.max_row):
    cell = ws.cell(row=row, column=total_amount_col)
    next_cell = ws.cell(row=row + 1, column=total_amount_col)
    
    # Check if this row has a numeric total and the next row starts a new group (or is blank)
    if isinstance(cell.value, (int, float)) and (next_cell.value is None or not isinstance(next_cell.value, (int, float))):
        cell.fill = yellow_fill

# Save the modified file
output_file = "highlighted_subtotals.xlsx"


# Define basic thin border style
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin")
)

# Include the header row in the first section’s border
start_row = 1
previous_value = None

# Loop through rows to detect section changes
for row in range(2, ws.max_row + 1):
    current_value = ws[f"A{row}"].value  # Adjust 'A' to the key column if needed

    # Check if the section changes or it's the last row
    if current_value != previous_value or row == ws.max_row:
        # Apply borders to the whole section, including the last row
        for r in range(start_row, row + 1):
            if any(ws.cell(row=r, column=c).value is not None for c in range(1, ws.max_column + 1)):
                for col in range(1, ws.max_column + 1):
                    cell = ws.cell(row=r, column=col)
                    cell.border = thin_border

        # Start the next section after the blank row
        start_row = row + 1

    previous_value = current_value

# Define the columns to format
amount_col = list(df.columns).index('Amount') + 1
total_amount_col = list(df.columns).index('Total_Amount') + 1

# Loop through rows and apply pound formatting to numeric cells
for row in range(2, ws.max_row + 1):  # Skip header row
    for col in [amount_col, total_amount_col]:
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.number_format = '£#,##0.00' 


# Save the updated file
wb.save('final_output_with_totals.xlsx')